In [2]:
# coding=utf-8
# import sys
# sys.path.append('..')

import os
import io
import random
import copy 
import re
import html
import numpy as np

from time import time 
from multiprocessing import Pool
from collections import Counter

from functools import partial

from tqdm import tqdm_notebook
import pythainlp
from pythainlp.util import *
from pythainlp.tokenize import word_tokenize
from pythainlp.ulmfit import *

# subword-nmt
from subword_nmt import learn_bpe as learner
from subword_nmt import apply_bpe as subword_tokenizer

import fairseq 
from datetime import timedelta
from tqdm import tqdm, tqdm_notebook
from pythainlp.tokenize import DEFAULT_DICT_TRIE

from pythainlp.corpus import thai_words

print(pythainlp.__version__)
# assert pythainlp.__version__ == '2.1'



import tensorflow as tf
import tensorflow_hub as hub
import sentencepiece as spm
import tf_sentencepiece 

2.0.7


/Users/saiko/miniconda3/envs/saiko/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/saiko/miniconda3/envs/saiko/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/saiko/miniconda3/envs/saiko/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/saiko/miniconda3/envs/saiko/lib/python3

In [3]:
# # install BPEmb (BPE embeddings)

# !pip install --q bpemb emoji subword-nmt fairseq tensorflow_hub sentencepiece tf_sentencepiece

In [4]:
# Universal Sentence Encoder

g = tf.Graph()
with g.as_default():
    text_input = tf.placeholder(dtype=tf.string, shape=[None])
    embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-multilingual/1")
    embedded_text = embed(text_input)
    init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [5]:


def compute_similarity(src_sent, tgt_sent):
    """
        Calculate sentence similarity based on Google Universal Sentence Encoder (Multilingual Large)
    """
    session = tf.Session(graph=g)
    session.run(init_op)
    
    src_result = session.run(embedded_text, feed_dict={text_input: [src_sent]})
    tgt_result = session.run(embedded_text, feed_dict={text_input: [tgt_sent]})
    
    return np.inner(src_result, tgt_result).reshape(1)[0]

def compute_similarity_list(src_sent, tgt_sent):
    """
        Calculate sentence similarity based on Google Universal Sentence Encoder (Multilingual Large)
    """
    session = tf.Session(graph=g)
    session.run(init_op)
    
    src_results = session.run(embedded_text, feed_dict={text_input: src_sent})
    tgt_results = session.run(embedded_text, feed_dict={text_input: tgt_sent})

    similarities = []
    for index, src_result in enumerate(src_results):
        similarity = np.inner(src_results[index], tgt_results[index]).reshape(1)[0]
        similarities.append(similarity)

    return similarities


In [6]:
print(compute_similarity("ฉันชื่อยีน","My Name is Gene"))
# print(compute_similarity("ฉันชื่อยีน","My Name is Jane"))
print(compute_similarity("ฉันชื่อยีน","My Name is Jeans"))
# print(compute_similarity("ฉันชื่อยีน","My Name is Jean"))
print(compute_similarity_list(["ฉันชื่อยีน", "ฉันชื่อยีน"], ["My Name is Gene", "My Name is Jeans"]))

0.8449457
0.7397201
[0.8449458, 0.7397201]


In [7]:
from bpemb import BPEmb

bpemb_pretrained ={
    'th': {
        '25000': BPEmb(lang="th", vs=25000)
    },
    'en': {
        '25000': BPEmb(lang="en", vs=25000)
    }
}


In [8]:
with open('../data/opensubtitle_v2018/OpenSubtitles.en-th.en','r', encoding='utf-8') as f:
    en = f.read().split('\n')
len(en),en[:3]


(3281534,
 ['Slave in the Magic Mirror, come from the farthest space.',
  'Through wind and darkness, I summon thee.',
  'Speak!'])

In [9]:
with open('../data/opensubtitle_v2018/OpenSubtitles.en-th.th','r', encoding='utf-8') as f:
    th = f.read().split('\n')
    
len(th),th[:3]

(3281534,
 ['ทาสในกระจกวิเศษ, มาจากพื้นที่ที่ไกลที่สุด',
  'ผ่านลมและความมืดฉันเรียกเจ้า',
  'พูด!'])

# Preprocess Opensubtitles_v2018


## 1.1 Text cleaning, filtering out

In [10]:
import sys
sys.path.append('..')

In [11]:
from mt_opus.preprocess import (
    SentenceLengthLessThanOrEqualToOne,
    SentenceContainsUnknownSymbol,
    SentenceContainsAdSymbol,
    ThaiSentenceContainsNoThaiCharacters,
    ThaiSentenceContainsNoThaiCharactersPattern,
    ThaiSentenceContainsUnwantedPattern,
    EnglishSentenceContainsUnwantedPattern,
    SentenceContainsOnlyAsterisk,
    
    UnescapeString,
    RemoveUnwantedSymbols,
    RemoveUnwantedPattern,
    ReplaceDashInSentence,
    RemoveHashtagInSentence,
    RemoveFullStopInThaiSentence,
    NormalizeThaiVowel,
    ReplaceAsteriskInSentence,
    RemoveColonInSentence,
    RemoveSemiColonInSentence,
    FormatTime,
    
    SentencePairFoundRepeatedText,
    SentencePairTokenLengthsDifferGreaterThreashold,
)

In [12]:
# to filter out a sentence pair
filtering_rules = [
    SentenceLengthLessThanOrEqualToOne(),
    SentenceContainsAdSymbol(),
    SentenceContainsUnknownSymbol(),
    ThaiSentenceContainsNoThaiCharacters(),
    ThaiSentenceContainsNoThaiCharactersPattern(),
    SentenceContainsOnlyAsterisk(),
]

cleaning_rules = [
    UnescapeString(),
    RemoveUnwantedSymbols(),
    RemoveUnwantedPattern(),
    ReplaceDashInSentence(),
    RemoveHashtagInSentence(),
    RemoveFullStopInThaiSentence(),
    NormalizeThaiVowel(),
    ReplaceDashInSentence(),
    ReplaceAsteriskInSentence(),
    RemoveColonInSentence(),
    RemoveSemiColonInSentence(),
    FormatTime(),
]

filtering_sentence_pair_rules = [
    SentencePairFoundRepeatedText(),
    SentencePairTokenLengthsDifferGreaterThreashold(threshold=0.85),
]

filtering_rules_post = [
    SentenceLengthLessThanOrEqualToOne(),
    ThaiSentenceContainsUnwantedPattern(),
    EnglishSentenceContainsUnwantedPattern(),
]

In [13]:
# this could be parallelized
def filter_sentence(sentence, lang, rules=filtering_rules):
    """
        Return True if a sentence match filtering pattern
    """
    for rule in rules:
        rule_obj = rule
        if rule_obj.test(sentence, lang=lang):
            return True
    return False

def filter_sentences(sentences, lang, rules=filtering_rules):
    """
        Returns a list of Boolean value, if such element is True, it means filter that sentence pair.
        Otherwise, keep that sentnence pair.
    """
    filtering_indices = []
    p = Pool() # use all available cores
    t = time()

    _filter_sentence = partial(filter_sentence, lang=lang, rules=rules)
    filtering_indices = p.map(_filter_sentence, sentences)
    
    p.close()
    p.join() # call Pool.join() to wait for the worker processes to terminate.

    filtering_indices_np = np.array(filtering_indices)
    number_of_filtered_out = np.sum(filtering_indices_np)
    
    print('Time taken: {} s'.format(time() -t))
    print('# sentences ({}) before filtered out'.format(lang), len(sentences))
    print('# sentences ({}) filtered out'.format(lang), number_of_filtered_out)
    print('# sentences ({}) after filtered out'.format(lang), len(sentences) - number_of_filtered_out)

    return filtering_indices_np, number_of_filtered_out


def filter_sentence_pair(sentence_pair, rules=filtering_sentence_pair_rules):
    """
        Return True if a sentence match filtering sentence pair pattern
    """
    for rule in rules:
        rule_obj = rule
        if rule_obj.test(sentence_pair):
            return True
    return False

def filter_sentence_pairs(sentence_pairs, rules=filtering_sentence_pair_rules):
    """
        Returns a list of Boolean value, if such element is True, it means filter that sentence pair.
        Otherwise, keep that sentnence pair.
    """
    filtering_indices = []
    p = Pool() # use all available cores
    t = time()

    _filter_sentence_pair = partial(filter_sentence_pair, rules=rules)
    filtering_indices = p.map(_filter_sentence_pair, sentence_pairs)
    
    p.close()
    p.join() # call Pool.join() to wait for the worker processes to terminate.

    filtering_indices_np = np.array(filtering_indices)
    number_of_filtered_out = np.sum(filtering_indices_np)
    
    print('Time taken: {} s'.format(time() -t))
    print('# sentences before filtered out', len(sentence_pairs))
    print('# sentences filtered out', number_of_filtered_out)
    print('# sentences after filtered out', len(sentence_pairs) - number_of_filtered_out)

    return filtering_indices_np, number_of_filtered_out


def clean_sentence(sentence, lang, rules=cleaning_rules):
    for rule in rules: 
        rule_obj = rule
        if rule_obj.test(sentence, lang=lang):
            sentence = rule_obj.replace(sentence, lang=lang)
    return sentence

def clean_sentences(sentences, lang, rules=cleaning_rules):
    """
        Clean the sentence with the specified text cleaning rules
        Return a list of cleaned sentences
    """
    p = Pool() # use all available cores
    t = time()
    
    _clean_sentence = partial(clean_sentence, lang=lang, rules=rules)
    cleaned_sentences = p.map(_clean_sentence, sentences)
    
    p.close()
    p.join() # call Pool.join() to wait for the worker processes to terminate.

    print('Time taken: {} s'.format(time() -t))
    
    return cleaned_sentences


### 1) Filter by each language 

In [14]:
filtering_indices_th, _ = filter_sentences(th, lang='th')

Time taken: 23.54386019706726 s
# sentences (th) before filtered out 3281534
# sentences (th) filtered out 77301
# sentences (th) after filtered out 3204233


In [15]:
filtering_indices_en, _ = filter_sentences(en, lang='en')

Time taken: 5.6949920654296875 s
# sentences (en) before filtered out 3281534
# sentences (en) filtered out 4577
# sentences (en) after filtered out 3276957


In [16]:
filtering_indices_th_en = filtering_indices_th | filtering_indices_en

print('# filtering_indices_th_en', sum(filtering_indices_th_en))

filtered_th = [th[i] for i, filtered_out in enumerate(filtering_indices_th_en) if not filtered_out]
filtered_en = [en[i] for i, filtered_out in enumerate(filtering_indices_th_en) if not filtered_out]

# filtering_indices_th_en 79608


In [17]:
print('# filtered_th', len(filtered_th))
print('# filtered_en', len(filtered_en))

# filtered_th 3201926
# filtered_en 3201926


In [18]:
3281534 - 78930

3202604

### 2) Filter by pair of source and target language

In [19]:

th_en_tuples = [(filtered_th[i], filtered_en[i]) for i in range(0, len(filtered_th))]
filtering_pairs_indices_th_en, _ = filter_sentence_pairs(th_en_tuples)

filtered_pairs_th = [filtered_th[i] for i, filtered_out in enumerate(filtering_pairs_indices_th_en) if not filtered_out]
filtered_pairs_en = [filtered_en[i] for i, filtered_out in enumerate(filtering_pairs_indices_th_en) if not filtered_out]
sentence_paris_filted_out = [th_en_tuples[i] for i, filtered_out in enumerate(filtering_pairs_indices_th_en) if filtered_out]



Time taken: 142.86196208000183 s
# sentences before filtered out 3201926
# sentences filtered out 6985
# sentences after filtered out 3194941


In [20]:
# th_en_tuples

In [21]:
print('# filtered_pairs_th', len(filtered_pairs_th))
print('# filtered_pairs_en', len(filtered_pairs_en))

# filtered_pairs_th 3194941
# filtered_pairs_en 3194941


In [22]:
for sent in filtered_pairs_en:
    if len(sent) <= 1:
        print(sent)

### 3) Text cleaning

In [23]:
cleaned_th = clean_sentences(filtered_pairs_th, lang="th")

Time taken: 104.53533387184143 s


In [24]:
cleaned_en = clean_sentences(filtered_pairs_en, lang="en")

Time taken: 69.936194896698 s


In [25]:
print('cleaned_th', len(cleaned_th))

cleaned_th 3194941


#### Explore cleaned sentences

In [26]:
# c = Counter()
# for i, sent in enumerate(cleaned_th):
#     if len(sent) <= 1:
#         c['found'] += 1
#         if c['found'] <= 15:
#             print(sent,'<>', cleaned_en[i])
#             print()
# c

### 4) Post-filtering out rule

- redo the filtering rules for 1) each langauge individually and 2) a language pair again.

In [27]:
filtering_post_indices_th, _ = filter_sentences(cleaned_th, lang='th', rules=filtering_rules_post)
filtering_post_indices_en, _ = filter_sentences(cleaned_en, lang='en', rules=filtering_rules_post)

filtering_post_indices_th_en = filtering_post_indices_th | filtering_post_indices_en


cleaned_and_filtered_th = [cleaned_th[i] for i, filtered_out in enumerate(filtering_post_indices_th_en) if not filtered_out]
cleaned_and_filtered_en = [cleaned_en[i] for i, filtered_out in enumerate(filtering_post_indices_th_en) if not filtered_out]


Time taken: 5.934873104095459 s
# sentences (th) before filtered out 3194941
# sentences (th) filtered out 111
# sentences (th) after filtered out 3194830
Time taken: 5.803123950958252 s
# sentences (en) before filtered out 3194941
# sentences (en) filtered out 277
# sentences (en) after filtered out 3194664


In [28]:
th_en_tuples = [(cleaned_and_filtered_th[i], cleaned_and_filtered_en[i]) for i in range(0, len(cleaned_and_filtered_th))]
filtering_pairs_indices_th_en, _ = filter_sentence_pairs(th_en_tuples)

cleaned_and_filtered_th = [cleaned_and_filtered_th[i] for i, filtered_out in enumerate(filtering_pairs_indices_th_en) if not filtered_out]
cleaned_and_filtered_en = [cleaned_and_filtered_en[i] for i, filtered_out in enumerate(filtering_pairs_indices_th_en) if not filtered_out]
sentence_paris_filted_out = [th_en_tuples[i] for i, filtered_out in enumerate(filtering_pairs_indices_th_en) if filtered_out]



Time taken: 157.341943025589 s
# sentences before filtered out 3194570
# sentences filtered out 1200
# sentences after filtered out 3193370


In [25]:
# After 4 steps

# number of sentence pairs is 3,201,120
# number of sentence pairs is 3,200,973 (Add SentenceContainsAsterink rule)
# number of sentence pairs is 3,194,664(Add SentencePairTokenLengthsDifferGreaterThreashold rule)


In [29]:
# c = Counter()
# for i, sent in enumerate(cleaned_and_filtered_th):

#     if re.search(r"--", sent):
#         c['count'] += 1
#         if c['count'] <= 20:
#             print(cleaned_and_filtered_th[i])
#             print(cleaned_and_filtered_en[i])
#             print('')

# print(c)

In [30]:
# test rules

In [31]:
c = Counter()
_tokenize = partial(word_tokenize, engine="newmm", keep_whitespace=False)

threashold = 0.85
for i, sent in tqdm_notebook(enumerate(cleaned_and_filtered_th), total=len(cleaned_and_filtered_th)):
    
    src_toks, tgt_toks = _tokenize(cleaned_and_filtered_th[i]), _tokenize(cleaned_and_filtered_en[i])
    src_toks_len, tgt_toks_len = len(src_toks), len(tgt_toks)
        
    diff = abs(src_toks_len- tgt_toks_len )
    diff_ratio = diff / ( max(src_toks_len, tgt_toks_len) ) 

    if  diff_ratio >= threashold:
        c['counter_{:.2}'.format(threashold)] += 1

        if c['counter_{:.2}'.format(threashold)] <= 200:

            print(sent, '---', cleaned_and_filtered_en[i], ' --- diff/ diff_ratio:', diff, diff_ratio)
            print()
print('count:', c['counter_{:.2}'.format(threashold)])


In [36]:
c = Counter()
t = time()
threashold = 0.25
step = 10000

counter = 0
for n in tqdm_notebook(range(0, len(cleaned_and_filtered_th), step)):
    similarities = compute_similarity_list(cleaned_and_filtered_th[n:n + step], cleaned_and_filtered_en[n:n + step])

    for i, sent in enumerate(cleaned_and_filtered_th[n:n+step]):
        src_sent, tgt_sebt = cleaned_and_filtered_th[i], cleaned_and_filtered_en[i]

        similariry = similarities[i]
        if similariry < threashold:
            counter += 1
            
print('count: {}'.format(counter))
print('{}'.format(time() - t ))

ฉันหวังว่าฉันหวัง 
 I'm hoрing -I'm hoрing 
 similariry 0.20642447 

จมูกของฉัน! 
 Take them away. 
 similariry 0.10670066 

ลัปล แดปลินส 
 Lumple dapplins? 
 similariry 0.15752248 

ทำไมเงียบไป? 
 Cat got your tongue? 
 similariry 0.2079348 

ไม่ไม่ไม่ไม่ได้รับความตื่นเต้น! 
 That's the tub. 
 similariry 0.060022634 

เดอะควีนเก่ากลับกลอกหนึ่ง 
 The... 
 similariry 0.041958585 

เคลีย พื้นที่ 
 Stand clear! 
 similariry 0.14087138 

เคลีย พื้นที่ 
 Stand clear. 
 similariry 0.20175225 

ครับผม ตรวจฉนวน 
 Yes, the fuse. 
 similariry 0.19460742 

ให้ตายสิ เผ่นเร็ว 
 Look out! Run! 
 similariry 0.14973816 

เครื่องบิน ข้าศึก 
 Air raid. After Big Bertha. 
 similariry 0.19183967 

ขอบใจหลาย ในความกรุณา โทไมเนีย ภูมิใจที่มีนาย 
 Thank you. I'll see you get the Tomainian Cross for this. 
 similariry 0.21292156 

ผมเต็มใจช่วย ครับผม 
 Only too willing to oblige, sir. 
 similariry 0.24054368 

โอ้ย.. เรื่องขี้หมา 
 I can try. 
 similariry 0.067670666 

จะต้องไปส่งเอกสารสำคัญ 
 Hold on to thes

In [30]:
with open('../data/opensubtitle_v2018/sent.cleaned.v1.th', 'w') as f:
    for sent in cleaned_and_filtered_th:
        f.write(sent + '\n')
        
with open('../data/opensubtitle_v2018/sent.cleaned.v1.en', 'w') as f:
    for sent in cleaned_and_filtered_en:
        f.write(sent + '\n')



In [ ]:
# Split sentence
N = len(cleaned_and_filtered_th)

print('N = ',N)
idx = list(range(N))

random.seed(1234) # Set SEED
random.shuffle(idx)

train_idx, test_idx = idx[:int(n*0.8)], idx[int(n*0.9):]

dataset_split = {}
cleaned_and_filtered_th_train = [cleaned_and_filtered_th[i] for i in train_idx]
cleaned_and_filtered_th_test = [cleaned_and_filtered_th[i] for i in test_idx]
cleaned_and_filtered_en_train = [cleaned_and_filtered_en[i] for i in train_idx]
cleaned_and_filtered_en_test =[cleaned_and_filtered_en[i] for i in test_idx]

In [ ]:
with open('../data/opensubtitle_v2018/sent.cleaned.v1.train.th', 'w') as f:
    for sent in cleaned_and_filtered_th_train:
        f.write(sent + '\n')
with open('../data/opensubtitle_v2018/sent.cleaned.v1.test.th', 'w') as f:
    for sent in cleaned_and_filtered_th_test:
        f.write(sent + '\n')     
with open('../data/opensubtitle_v2018/sent.cleaned.v1.train.en', 'w') as f:
    for sent in cleaned_and_filtered_en_train:
        f.write(sent + '\n')
with open('../data/opensubtitle_v2018/sent.cleaned.v1.test.en', 'w') as f:
    for sent in cleaned_and_filtered_en_test:
        f.write(sent + '\n')



## 1.2 Text Segmentation (word, subword)


In [31]:
# define functions for newmm tokenization

def tokenize_worker(sentence, lang, trie):
    
    _tokenizer_newmm = partial(pythainlp.tokenize.word_tokenize, engine='newmm',
                               keep_whitespace=False,
                              custom_dict=(trie if trie != None else DEFAULT_DICT_TRIE))
    return ' '.join(_tokenizer_newmm(sentence))
  
def tokenize_handler(sentences, lang, trie=None):
    toks = []
    p = Pool(6)
    t = time()
    _tokenize_worker = partial(tokenize_worker, lang=lang, trie=trie)
    toks = p.map(_tokenize_worker, sentences)
    
    p.close()
    p.join() # call Pool.join() to wait for the worker processes to terminate.

    print('{} s'.format(time() -t))

    return toks
  

In [32]:
def write_spaced_tokens_to_file(data, folder_name, filename):
    with open('/root/mt-opus/data/{}/{}'.format(folder_name, filename),'w') as f:
        for item in data:
            f.write(item + '\n')
            
            

In [33]:
toks = {
    'th': {
        'sentencepiece': [],
        'newmm':[]
    },
    'en': {
        'sentencepiece': [],
        'newmm':[]
    }
}

### 1a) Segment text into words with a dictionary-based tokenizer (`newmm`)

In [34]:
# Thai
toks['th']['newmm'] = tokenize_handler(filtered_th, lang='th')
# English
toks['en']['newmm'] = tokenize_handler(filtered_en, lang='en')


80.45962905883789 s
63.490153074264526 s


In [107]:
toks['th']['newmm'][0:10], toks['en']['newmm'][0:10]

(['ทาส ใน กระจก วิเศษ , มาจาก พื้นที่ ที่ ไกล ที่สุด',
  'ผ่าน ลม และ ความมืด ฉัน เรียก เจ้า',
  'พูด !',
  'ให้ ฉัน เห็น พระพักตร์ ของ พระองค์',
  'สิ่ง ที่ เจ้า จะ รู้ ว่า สมเด็จ พระราชินี ของ ฉัน ได้ อย่างไร',
  'กระจก วิเศษ บน ผนัง ผู้ ที่ เป็น สังขาร หนึ่ง ทั้งหมด หรือไม่',
  'ที่ มีชื่อเสียง เป็น ความงาม ของ เจ้า พระ บาท สมเด็จ พระเจ้าอยู่หัว',
  'แต่ ถือเป็น แม่บ้าน ที่ น่ารัก ที่ ฉัน เห็น',
  'ยาจก ไม่ สามารถ ซ่อน พระคุณ อ่อนโยน ของ เธอ',
  'อนิจจา เธอ มี ความเป็นธรรม มากขึ้น กว่า เจ้า'],
 ['Slave in the Magic Mirror , come from the farthest space .',
  'Through wind and darkness , I summon thee .',
  'Speak !',
  'Let me see thy face .',
  'What wouldst thou know , my Queen ?',
  'Magic Mirror on the wall , who is the fairest one of all ?',
  'Famed is thy beauty , Majesty .',
  'But hold , a lovely maid I see .',
  'Rags cannot hide her gentle grace .',
  'Alas , she is more fair than thee .'])

### 1b) Segment text into subword (ie. BPE tokens) with Pretrained SentencePiece (`BPEmb`)


In [108]:
def encode_bpe(sentences, lang, n_vocab=25000):
    """Return a list of bpe tokens give a list of sentences"""
    segmented_sentences = []
    for sentence in tqdm_notebook(sentences, total=len(sentences)):
#         print(sentence)
        bpe_tokens = bpemb_pretrained[lang]['{}'.format(n_vocab)].encode(sentence)
        segmented_sentences.append(' '.join(bpe_tokens))
        
    return segmented_sentences

__Thai language__

In [109]:
toks['th']['sentencepiece'] = encode_bpe(filtered_th, 'th', 25000)

print(toks['th']['sentencepiece'][0:10])


['▁ท าส ใน กระจก วิเศษ , ▁มาจาก พื้นที่ ที่ ไกล ที่สุด', '▁ผ่าน ลม และความ มืด ฉัน เรียก เจ้า', '▁พูด !', '▁ให้ ฉัน เห็น พระพักตร์ ของ ▁พระองค์', '▁สิ่งที่ เจ้า จะ รู้ว่า สมเด็จพระราชินี ▁ของ ฉัน ได้อย่างไร', '▁กระจ ก วิเศษ บน ผนัง ▁ผู้ ที่เป็น สัง ขาร หนึ่ง ทั้งหมด ▁หรือไม่', '▁ที่มีชื่อเสียง เป็น ความงาม ของ ▁เจ้า พระบาทสมเด็จพระ เจ้าอยู่หัว', '▁แต่ ถือเป็น แม่ บ้าน ที่น ่ารัก ที่ ฉัน ▁เห็น', '▁ยา จก ไม่สามารถ ซ่อน พระคุณ ▁อ่อน โยน ของเธอ', '▁อน ิจ จา เธอ มีความเป็น ธรรม ▁มาก ขึ้น กว่า เจ้า']


__English language__

In [110]:
toks['en']['sentencepiece']  = encode_bpe(filtered_en, 'en', 25000)
print(toks['en']['sentencepiece'][0:10])


['▁slave ▁in ▁the ▁magic ▁mirror , ▁come ▁from ▁the ▁fart hest ▁space .', '▁through ▁wind ▁and ▁darkness , ▁i ▁summon ▁the e .', '▁speak !', '▁let ▁me ▁see ▁thy ▁face .', '▁what ▁would st ▁thou ▁know , ▁my ▁queen ?', '▁magic ▁mirror ▁on ▁the ▁wall , ▁who ▁is ▁the ▁fa ire st ▁one ▁of ▁all ?', '▁famed ▁is ▁thy ▁beauty , ▁majesty .', '▁but ▁hold , ▁a ▁lov ely ▁maid ▁i ▁see .', '▁ra gs ▁cannot ▁hide ▁her ▁gentle ▁grace .', '▁al as , ▁she ▁is ▁more ▁fair ▁than ▁the e .']


### 1c) Segment text into subword (ie. BPE tokens) with SentencePiece model (unigram) trained from Opensubtitles_v2018 dataset

### Build SentencePiece BPE 

In [48]:

train_file = '../data/opensubtitle_v2018/sent.cleaned.v1.train'
vocab_file = '../data/opensubtitle_v2018/opensubtitles.vocab'
!cat {train_file}.th {train_file}.en | shuffle > train
!spm_train --input=train --model_prefix=spm --vocab_size=8000 --character_coverage=0.9995 --normalization_rule_name identity
!spm_encode --model=spm.model --generate_vocabulary < {train_file}.L1 > {vocab_file}.th
!spm_encode --model=spm.model --generate_vocabulary < {train_file}.L2 > {vocab_file}.en

ERROR: --input must not be empty

sentencepiece

Usage: sentencepiece [options] files

   --accept_language (comma-separated list of languages this model can accept)  type: string  default: 
   --add_dummy_prefix (Add dummy whitespace at the beginning of text)  type: bool  default: true
   --bos_id (Override BOS (<s>) id. Set -1 to disable BOS.)  type: int32  default: 1
   --bos_piece (Override BOS (<s>) piece.)  type: string  default: <s>
   --character_coverage (character coverage to determine the minimum symbols)  type: double  default: 0.9995
   --control_symbols (comma separated list of control symbols)  type: string  default: 
   --eos_id (Override EOS (</s>) id. Set -1 to disable EOS.)  type: int32  default: 2
   --eos_piece (Override EOS (</s>) piece.)  type: string  default: </s>
   --hard_vocab_limit (If set to false, --vocab_size is considered as a soft limit.)  type: bool  default: true
   --input (comma separated list of input sentences)  type: string  default: 
   --input

### 2) Split train-valid-test 

- split: train/valid/test (80/10/10)

- seed for rando,.shuffle: 1234


In [111]:
#train-valid-test split 80/10/10

n = len(toks['th']['newmm'])

print('N = ',n)
idx = list(range(n))

random.seed(1234) # Set SEED
random.shuffle(idx)

train_idx, valid_idx, test_idx = idx[:int(n*0.8)], idx[int(n*0.8):int(n*0.9)], idx[int(n*0.9):]

dataset_split = {}
dataset_split['train'] = train_idx
dataset_split['valid'] = valid_idx
dataset_split['test'] = test_idx


len(train_idx),len(valid_idx),len(test_idx)



N =  3202751


(2562200, 320275, 320276)

In [112]:
dataset = {
    'train': {
        'en': {
            'sentencepiece': [],
            'newmm':[]
        },
        'th': {
             'sentencepiece': [],
            'newmm':[]
        }
    },
    'valid': {
        'en': {
            'sentencepiece': [],
            'newmm':[]
        },
        'th': {
             'sentencepiece': [],
            'newmm':[]
        }
    },
    'test': {
        'en': {
            'sentencepiece': [],
            'newmm':[]
        },
        'th': {
             'sentencepiece': [],
            'newmm':[]
        }
    }
}

for split_name in ['train', 'valid', 'test']:
    for lang in ['th', 'en']:
        for tok_type in ['sentencepiece', 'newmm']:

            dataset[split_name][lang][tok_type] = [toks[lang][tok_type][i] for i in dataset_split[split_name]] 


In [113]:
print(dataset['train']['th']['newmm'][0:2],'\n')
print(dataset['train']['en']['newmm'][0:2],'\n')
print(dataset['train']['th']['sentencepiece'][0:2],'\n')
print(dataset['train']['en']['sentencepiece'][0:2],'\n')

['เบค กี้ เธอ ทำท่า แปลก ๆ เมื่อกี้ ใน ห้อง', 'อยู่ กับ เธอ แอน นา จะ นำทาง คุณ ผม จะ กลับ ไป'] 

['Becky , um , you were acting particularly strange in there just now .', "Stay with her so Anna can guide you . I ' m going back ."] 

['▁เบ ค กี้ ▁เธอ ทํา ท่า แปลก ๆ ▁เมื่อ กี้ ▁ในห้อง', '▁ อยู่กับ เธอ ▁แอนนา จะนํา ทาง คุณ ▁ผม จะ กลับไป'] 

['▁bec ky , ▁um , ▁you ▁were ▁acting ▁particularly ▁strange ▁in ▁there ▁just ▁now .', "▁stay ▁with ▁her ▁so ▁anna ▁can ▁guide ▁you . ▁i ' m ▁going ▁back ."] 



In [119]:
# Counting number of tokens for train, valid, test
counter = Counter( )
for dataset_type in ['train', 'valid', 'test']:
    for th_sent_toks in dataset[dataset_type]['th']['newmm']:
        counter['th_{}_n_toks'.format(dataset_type)] += len(th_sent_toks)
    for en_sent_toks in dataset[dataset_type]['en']['newmm']:
        counter['en_{}_n_toks'.format(dataset_type)] += len(en_sent_toks)

print(counter) 

Counter({'en_train_n_toks': 92383739, 'th_train_n_toks': 86683223, 'en_valid_n_toks': 11536351, 'en_test_n_toks': 11535798, 'th_test_n_toks': 10833242, 'th_valid_n_toks': 10826042})


__Write segmented text to files__

In [124]:

for tok_type_src in ['sentencepiece', 'newmm']:
    for tok_type_tgt in ['sentencepiece', 'newmm']:
        langs = ['th', 'en']
        for lang in langs:
            src_lang = lang
            tgt_lang = 'en' if lang =='th' else 'th'
            FOLDER_NAME = "opensubtitles_tok/{}-{}/{}-{}".format(tok_type_src, tok_type_tgt, src_lang, tgt_lang )
            FOLDER_NAME_BIN = "opensubtitles_bin/{}-{}/{}-{}".format(tok_type_src, tok_type_tgt, src_lang, tgt_lang)
           
            
            # Create directories
            print('create directories: ')
            print('dir: ../data/{}'.format(FOLDER_NAME))
            print('dir: ../data/{}'.format(FOLDER_NAME_BIN))

            !mkdir -p ../data/{FOLDER_NAME}
            !mkdir -p ../data/{FOLDER_NAME_BIN}

            for split_name in ['train', 'valid', 'test']:
                
                write_spaced_tokens_to_file(dataset[split_name][src_lang][tok_type_src],
                                            FOLDER_NAME, '{}.{}'.format(split_name, src_lang))
                
                write_spaced_tokens_to_file(dataset[split_name][tgt_lang][tok_type_tgt],
                                            FOLDER_NAME, '{}.{}'.format(split_name, tgt_lang))


create directories: 
dir: ../data/opensubtitles_tok/sentencepiece-sentencepiece/th-en
dir: ../data/opensubtitles_bin/sentencepiece-sentencepiece/th-en
create directories: 
dir: ../data/opensubtitles_tok/sentencepiece-sentencepiece/en-th
dir: ../data/opensubtitles_bin/sentencepiece-sentencepiece/en-th
create directories: 
dir: ../data/opensubtitles_tok/sentencepiece-newmm/th-en
dir: ../data/opensubtitles_bin/sentencepiece-newmm/th-en
create directories: 
dir: ../data/opensubtitles_tok/sentencepiece-newmm/en-th
dir: ../data/opensubtitles_bin/sentencepiece-newmm/en-th
create directories: 
dir: ../data/opensubtitles_tok/newmm-sentencepiece/th-en
dir: ../data/opensubtitles_bin/newmm-sentencepiece/th-en
create directories: 
dir: ../data/opensubtitles_tok/newmm-sentencepiece/en-th
dir: ../data/opensubtitles_bin/newmm-sentencepiece/en-th
create directories: 
dir: ../data/opensubtitles_tok/newmm-newmm/th-en
dir: ../data/opensubtitles_bin/newmm-newmm/th-en
create directories: 
dir: ../data/opens

In [125]:
!head ../data/opensubtitles_tok/newmm-sentencepiece/th-en/train.en


▁bec ky , ▁um , ▁you ▁were ▁acting ▁particularly ▁strange ▁in ▁there ▁just ▁now .
▁stay ▁with ▁her ▁so ▁anna ▁can ▁guide ▁you . ▁i ' m ▁going ▁back .
▁look .
▁oh , ▁no , ▁it ' s ▁the ▁other ▁way ▁around , ▁dr . ▁lewis .
▁sort ▁of .
▁bart ender , ▁something ▁really ▁strong , ▁please .
▁yes , ▁obviously .
▁la ' s ▁so ▁nice .
▁i ' m ▁going ▁to ▁fix ▁it .
▁i ▁get ▁b ored .


In [126]:
!head ../data/opensubtitles_tok/newmm-sentencepiece/th-en/train.th

เบค กี้ เธอ ทำท่า แปลก ๆ เมื่อกี้ ใน ห้อง
อยู่ กับ เธอ แอน นา จะ นำทาง คุณ ผม จะ กลับ ไป
ฟัง นะ
พอดี เลย ดร. ลี วิ ส
แบบ ว่า
เอ่อ บาร์ เท็น เด อร ์ ขอ อะไร ที่
ก็ ใช่ ห น่ะ สิ
แอลเอ สวย เนอะ
ฉัน กำลังจะ แก้ ไขมัน
ฉัน เบื่อ ละ
